In [190]:
# Importo pandas para trabajar con dataframes, y numpy para posibles operaciones numéricas con los datos.

import pandas as pd
import numpy as np

In [191]:
dataset = pd.read_csv("Data_To_Hourervals_no_filter.csv", sep=';')

url_dataset = "https://www.kaggle.com/jaimebadiola/bitcoin-tweets-and-price/version/1"

In [192]:
dataset.head()

,Date,Compound_Score,n,Count_Negatives,Count_Positives,Count_Neutrals,Sent_Negatives,Sent_Positives,Open,High,Low,Close,Volume (BTC),Volume (Currency)
0,01/08/2017 0:00,0.097156,1027.0,148.0,403.0,476.0,-0.504061,0.432704,2855.81,2863.06,2823.00,2825.92,184.02,"522951,00"
1,01/08/2017 1:00,0.064507,778.0,143.0,239.0,396.0,-0.381007,0.437953,2823.01,2860.02,2821.01,2853.38,77.3,219605.16
2,01/08/2017 2:00,0.119218,836.0,118.0,333.0,385.0,-0.394999,0.439269,2846.27,2858.04,2837.31,2841.60,135.83,386739.15
3,01/08/2017 3:00,0.004163,984.0,262.0,279.0,443.0,-0.431913,0.420278,2841.84,2863.88,2837.73,2862.93,143.2,408360.03
4,01/08/2017 4:00,0.065608,751.0,133.0,237.0,381.0,-0.405835,0.435645,2862.92,2876.00,2848.11,2874.99,222.53,637045.88


In [193]:
# Muestro los nombres de las columnas de mi dataset, para ver si me puedo deshacer de alguna que no 
# tenga relevancia para mi análisis.

print(dataset.columns.values)

['Date' 'Compound_Score' 'n' 'Count_Negatives' 'Count_Positives'
 'Count_Neutrals' 'Sent_Negatives' 'Sent_Positives' 'Open' 'High' 'Low'
 'Close' 'Volume (BTC)' 'Volume (Currency)']


In [194]:
# Me interesan Date, Count_Negatives, Open, Close, y la columna que tenga menos duplicates para guardarla hasta el 
# final. # Miro a ver cuál es:


def less_duplicates(dataset):
    print("Dataset rows:", len(dataset.index))
    columnas = dataset.columns.values
    for a in dataset:
        values = []
        for v in dataset["{}".format(a)]:
            values.append(v)
        print("{}:".format(a), len(set(values)))

print(less_duplicates(dataset))

Dataset rows: 12936
Date: 12936
Compound_Score: 12934
n: 2913
Count_Negatives: 1152
Count_Positives: 1585
Count_Neutrals: 1755
Sent_Negatives: 12934
Sent_Positives: 12936
Open: 12676
High: 12322
Low: 12426
Close: 12674
Volume (BTC): 12876
Volume (Currency): 4591
None


In [195]:
# Me sirve Date. Por tanto, me deshago de todas las columnas que no necesito:

mis_columnas = ["Date", "Count_Negatives", "Close"]

for columna in dataset.columns.values:
    if columna not in mis_columnas:
        dataset = dataset.drop(columna, axis = 1)

In [196]:
dataset.head()

,Date,Count_Negatives,Close
0,01/08/2017 0:00,148.0,2825.92
1,01/08/2017 1:00,143.0,2853.38
2,01/08/2017 2:00,118.0,2841.60
3,01/08/2017 3:00,262.0,2862.93
4,01/08/2017 4:00,133.0,2874.99


In [197]:
# Compruebo a ver cuántos valores nulos tengo en cada columna:

null_cols = dataset.isnull().sum()

null_cols[null_cols > 0]

Count_Negatives    271
dtype: int64

In [198]:
len(dataset.index)

12936

In [199]:
# No me van a servir esos registros, así que me deshago de ellos:

dataset = dataset[dataset.Count_Negatives.notnull()]

In [200]:
null_cols = dataset.isnull().sum()

null_cols[null_cols > 0]

Series([], dtype: int64)

In [201]:
dataset.dtypes

Date                object
Count_Negatives    float64
Close              float64
dtype: object

In [202]:
# Quiero los valores de los datos en datetime, para poder categorizarlos en franjas temporales:

dataset["Date"] = pd.to_datetime(dataset["Date"])

In [203]:
dataset.head()

,Date,Count_Negatives,Close
0,2017-01-08 00:00:00,148.0,2825.92
1,2017-01-08 01:00:00,143.0,2853.38
2,2017-01-08 02:00:00,118.0,2841.60
3,2017-01-08 03:00:00,262.0,2862.93
4,2017-01-08 04:00:00,133.0,2874.99


In [204]:
dataset.dtypes

Date               datetime64[ns]
Count_Negatives           float64
Close                     float64
dtype: object

In [206]:
# Ahora quiero que en cada registro me aparezca el valor de Close del día siguiente

lista = [0]

lista += list(dataset["Close"])

lista.pop()
print(len(lista))
print(len(dataset["Close"]))

12665
12665


In [207]:
dataset["Close_Next_Day"] = lista

In [208]:
dataset.head()

,Date,Count_Negatives,Close,Close_Next_Day
0,2017-01-08 00:00:00,148.0,2825.92,0.00
1,2017-01-08 01:00:00,143.0,2853.38,2825.92
2,2017-01-08 02:00:00,118.0,2841.60,2853.38
3,2017-01-08 03:00:00,262.0,2862.93,2841.60
4,2017-01-08 04:00:00,133.0,2874.99,2862.93


In [209]:
dataset = dataset[dataset.Close_Next_Day != 0]

In [210]:
dataset.head()

,Date,Count_Negatives,Close,Close_Next_Day
1,2017-01-08 01:00:00,143.0,2853.38,2825.92
2,2017-01-08 02:00:00,118.0,2841.60,2853.38
3,2017-01-08 03:00:00,262.0,2862.93,2841.60
4,2017-01-08 04:00:00,133.0,2874.99,2862.93
5,2017-01-08 05:00:00,133.0,2900.00,2874.99


In [ ]:
# Ya tengo mi dataset limpio. Empiezo a hacer API requests:

